In [165]:
import logging
import numpy as np

import matplotlib.pyplot as plt
from importlib import reload
import random

import tensorflow as tf
import importlib
import MCAgent as agent
import Model as Con4
import All_In_One_environment as env
from tensorflow.python.client import device_lib
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten, BatchNormalization, Activation, LeakyReLU, add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

In [217]:
#local imports reload
importlib.reload(agent)
importlib.reload(Con4)
importlib.reload(env)

<module 'All_In_One_environment' from '/home/jevans40/AI/AIClass2021/Project/Group2-s_UW_Intro_AI_Project/Project_attempt_one/Project_attempt_one/All_In_One_environment.py'>

In [216]:
#settings:
#Settings
inputDim = (6,7,2)
outputDim = (7)
filterSize = 20
kernelDim = (4,4)
convLayers = 4
regConst = 0.0001
learningRate = 0.001
iteration = 1

Memory = []

In [218]:
def RunTwoAgents(Agent1,Agent2, Games, Verbosity = 0):
    memories = []
    for game in range(Games):
        print("Playing game {} out of {}".format(game,Games))
        winner = PlayGame(Agent1,Agent2,Verbosity)
        memories += Agent1.GetMemory()
        memories += Agent2.GetMemory()
        Agent1.ClearMemory()
        Agent2.ClearMemory()   

        tmp = Agent2
        Agent2 = Agent1
        Agent1 = tmp
    return memories
    #Return memory to train on

#Verbosity controls how much information to output
#0: None
#1: Winner/Loser
#2: Memory Output
def PlayGame(Agent1,Agent2,Verbosity=0):
    Agent1.SetPlayer(-1)
    Agent2.SetPlayer(1)
    done = False
    memories = []
    newGame = env.ConnectFour(1)
    num = random.randint(0,6)
    newGame.step(num,1)
    while not done:
        action = Agent1.GetAction(newGame)
        #Add some level of randomness so that we can get more varied cases
        nu = np.random.dirichlet([1]*7)/20
        action = action + nu
        newState,reward,done,info = newGame.step_array(action,Agent1.player)
        memories.append(action)
        if Verbosity > 1:
            print("Player -1 took the action\n{}".format(action))
        if done:
            if Verbosity > 0:
                print("Player -1 has won the game!")
            return -1

        action = Agent2.GetAction(newGame)
        nu = np.random.dirichlet([1]*7)/20
        action = action + nu
        newState,reward,done,info = newGame.step_array(action,Agent2.player)
        if Verbosity > 1:
            print("Player 1 took the action\n{}".format(action))
        if done:
            if Verbosity > 0:
                print("Player 1 has won the game!")
            return 1
        if Verbosity > 2:
            print(np.array(newGame.get_current_state()).reshape(6,7))
    
    #Return game result

def TrainModel(Memories, toFitAgent):
    for i in range(5):
        minbatch = random.sample(Memories,min(250,len(Memories)))

        trainingStates = np.array([Con4.ReshapeToModel(item[0],item[3]) for item in Memories])
        trainingTargets = np.array([item[1] for item in Memories])

        fit = toFitAgent.model.fit(trainingStates,trainingTargets,epochs=5,verbose=1,validation_split=0,batch_size = 32)

        toFitAgent.model.save("ModelCheckPointStart")
    return round(fit.history["loss"][0],4)

In [219]:
testModel = Con4.CN4Model(inputDim,outputDim,filterSize,kernelDim,regConst,convLayers,learningRate)
testModel.InitModel()


In [220]:
Agent1 = agent.ReinforcementAgent(testModel,-1)
Agent2 = agent.ReinforcementAgent(testModel,1)

In [ ]:
Mem = RunTwoAgents(Agent1, Agent2,5, Verbosity=2)
print(Mem)

In [ ]:
TrainModel(Mem,testModel)

In [221]:
while True:
    Mem = RunTwoAgents(Agent1, Agent2,7, Verbosity=0)
    TrainModel(Mem,testModel)

Playing game 0 out of 7
truth
[1.49820216e-04 2.49850180e-01 1.49820216e-04 1.49820216e-04
 3.74700360e-01 3.74700360e-01 1.49820216e-04]
prediction
[0.14349471 0.14271155 0.14305106 0.14244401 0.14317857 0.14256045
 0.14255959]
truth
[0.12562806 0.05026027 0.52758964 0.05654092 0.09422482 0.10050547
 0.04523575]
prediction
[0.14383125 0.14183418 0.14234129 0.14379562 0.14278926 0.14217936
 0.14322905]
truth
[4.78163851e-04 5.98182977e-01 4.78163851e-04 4.78163851e-04
 4.78163851e-04 3.98948040e-01 4.78163851e-04]
prediction
[0.14294091 0.14182179 0.14374234 0.14229815 0.14322881 0.14247139
 0.1434966 ]
truth
[0.55473595 0.05690365 0.0213442  0.21336523 0.0213442  0.02561134
 0.10668688]
prediction
[0.14531337 0.141029   0.14249644 0.14374398 0.1413014  0.14194436
 0.14417149]
truth
[0.125 0.125 0.125 0.125 0.125 0.125 0.125]
prediction
[0.1429392  0.14269933 0.14446516 0.14193307 0.1423187  0.1431837
 0.14246082]
truth
[0.22673425 0.36625551 0.12209331 0.06105276 0.08139961 0.08139961